In [1]:
import csv
from collections import deque
import elasticsearch
from elasticsearch import helpers


In [2]:
def readMovies():
    csvfile = open('ml-latest-small/movies.csv', 'r', encoding='utf-8', errors='ignore')
    reader = csv.DictReader( csvfile )
    titleLookup = {}
    for movie in reader:
        titleLookup[movie['movieId']] = movie['title']
    return titleLookup

def readRatings():
    csvfile = open('ml-latest-small/ratings.csv', 'r', encoding='utf-8', errors='ignore')
    titleLookup = readMovies()
    reader = csv.DictReader( csvfile )
    for line in reader:
        rating = {}
        rating['user_id'] = int(line['userId'])
        rating['movie_id'] = int(line['movieId'])
        rating['title'] = titleLookup[line['movieId']]
        rating['rating'] = float(line['rating'])
        rating['timestamp'] = int(line['timestamp'])
        yield rating



In [3]:
es = elasticsearch.Elasticsearch(["http://199.241.138.138:9200"], http_auth=('elastic', 'changeme'))

es.indices.delete(index="ratings",ignore=404)
deque(helpers.parallel_bulk(es,readRatings(),index="ratings"), maxlen=0)
es.indices.refresh()

<ipython-input-3-96fb20648c52>:1: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = elasticsearch.Elasticsearch(["http://199.241.138.138:9200"], http_auth=('elastic', 'changeme'))
<ipython-input-3-96fb20648c52>:3: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index="ratings",ignore=404)


ObjectApiResponse({'_shards': {'total': 32, 'successful': 21, 'failed': 0}})